In [1]:
import gc
import torch
from honeybee.loaders import Slide
from honeybee.models import UNI, TissueDetector
import numpy as np
import matplotlib.pyplot as plt
from umap import UMAP

slide = Slide(
    slide_image_path="/mnt/f/Projects/HoneyBee/tmp/sample_wsi.svs",
    tileSize=256,
    max_patches=1_000,
    visualize=False,
    tissue_detector=TissueDetector(model_path="/mnt/d/Models/TissueDetector/HnE.pt"),
)
patches = slide.load_patches_concurrently(target_patch_size=224)

uni = UNI(model_path="/mnt/d/Models/UNI/pytorch_model.bin")
embedding = uni.load_model_and_predict(patches=patches)
print(embedding.shape)

gc.collect()
torch.cuda.empty_cache()

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.23 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


torch.Size([245, 1024])


In [2]:
embedding_np = embedding.cpu().numpy() if isinstance(embedding, torch.Tensor) else embedding
patch_coords = slide.get_patch_coords()

# map the embeddings_3d to RGB values for visualization and then visualize it on top of the slide
def umap_reducer(x: np.ndarray, dims: int = 3, nns: int = 10) -> np.ndarray:
    """UMAP reduction of the input data."""
    reducer = UMAP(
        n_neighbors=nns,
        n_components=dims,
        metric="manhattan",
        spread=0.5,
        random_state=2,
    )
    reduced = reducer.fit_transform(x)
    reduced -= reduced.min(axis=0)
    reduced /= reduced.max(axis=0)
    return reduced

rgb_values = umap_reducer(embedding_np)

/home/aakash/miniconda3/envs/honeybee/lib/python3.11/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [8]:
print("embedding shape:", embedding_np.shape)
print("RGB values shape:", rgb_values.shape)
print("Patch coords shape:", patch_coords.shape)

embedding shape: (245, 1024)
RGB values shape: (245, 3)
Patch coords shape: (414, 2)
